<hr/>

<b>Notebook Summary</b>

The purpose of this notebook is a continuation from ES1_2, which introduced some basic properties of graphs and provide examples and proofs that demonstrate these properties. It will also introduce the "mutation game" which can be used to demonstrate these properties. 

These notes are based on Prof. Norman Wildberger's lectures on Dynamics on Graphs which can be found <a href="https://www.youtube.com/c/WildEggmathematicscourses/featured">here</a>. 
    
They notes are are being hosted at my website <a href="https://www.ladatavita.com/">ladatavita.com</a> and the Jupyter notebook is also available from my Github repo at: <a href="https://github.com/jgab3103/Jamie-Gabriel/tree/main/MathNotebooks">https://github.com/jgab3103/Jamie-Gabriel/tree/main/MathNotebooks</a>



<hr/>

In [1]:
import pyvis.network as nt
import numpy as np
import sympy as sp
from IPython.display import HTML
import ipywidgets as widgets
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
import pandas as pd
import networkx as nx
import string

<hr/>
<b>Aim</b>: Continue to demonstrate the properties of the mutation function on simple graphs
<hr/>

<b>Observe</b>: Recall that the population function (introduced in ES1_1), $ps_x$ has the following properties:  

1. If $p$ and $q$ are populations, $P(X)$ is a space of populations, and $p, q \in P(X)$, then $(p + q)s_x = ps_x + qs_x$ so it is a linear operator and can be applied pointwise and if $n \in \text{Int}$, then $(np)s_x = n(ps_x)$

2. $s_x^2 \equiv \text{Identity}$. Performing the same mutation in succession will produce the original graph population.

3. If $x$ and $y$ are non-neighboring vertices, then the $S_xS_y \equiv S_yS_x$, meaning they are commutative


<b>Observe</b>: A consequence of property $3$, it is the case that: 

$$ (pS_x)S_y = p(S_xS_y) \equiv (pS_y)S_x = p(S_yS_x) $$


Note that the mutation functino, $S_i$ is, by convention, written on the rhs of the populations it is are acting on. This means that
However in general they do not have to 

4. (The braid relation) If $x$ and $y$ are neighbors, then: 



$$ S_x S_y S_x = S_y S_x S_y $$

<b>Observe</b>: It can be shows that the relation $(S_xS_y)^3 = \text{identity}$



<b>Let</b> $F1$ be an implementation of the population function $ps_x$

In [2]:
def F1(graph, nodeChoice, printSummary = True, 
       returnUpdatedGraph = False, 
       returnListOfPopulations = False,
      returnAllAsDict = False):
   
    edgesOfChosenNode = list(nx.edges(graph, [nodeChoice]))
   
    neigborOfChosenNode = [edgesOfChosenNode[i][1] for i in range(len(list(edgesOfChosenNode)))]
    nodeChoicePopulation = graph.nodes[nodeChoice]['population']
    sumOfNeighborsOfChosenNode = np.sum([graph.nodes[i]['population'] for i in neigborOfChosenNode])
    populationOfNode = -nodeChoicePopulation + sumOfNeighborsOfChosenNode
    updatedGraph = graph.copy()
    updatedGraph.nodes[nodeChoice]['population'] = populationOfNode

    newPopulations = [updatedGraph.nodes[i]['population'] for i in list(updatedGraph)]
    if printSummary:
        print("Node choice", 
              nodeChoice,
              "\nNode details",
              nx.nodes(graph)[nodeChoice],
              "\nChange in node population ",
              nx.nodes(graph)[nodeChoice]['population'], 
              "->", 
              populationOfNode)
        print("Updated node populations of graph: ", newPopulations, "\n")

    if returnUpdatedGraph: 
        return(updatedGraph)
    
    if returnListOfPopulations:
        return(np.array(newPopulations))
    
    if returnAllAsDict:
        return({"graph": updatedGraph,
               "population": np.array(newPopulations)})

Suppose that x is a simple graph

Note that ~ makes things more symmetrical